In [209]:
import numpy as np
import pandas as pd

from ast import literal_eval
import os, time

# Random Functions

In [210]:
RESULT_DIR = "final_results"
benchmark = "manual"

In [211]:
df = {}
functions = None
for f_name in os.listdir(RESULT_DIR):
    try:
        benchmark_name = None
        if len(f_name.split("_")) == 4:
            benchmark_name = f_name.split("_")[1]
        elif len(f_name.split("_")) == 5:
            benchmark_name = "_".join(f_name.split("_")[1:-2])
                
        tool_name =  f_name.split("_")[-2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
            
            if tool_name == 'volesti':
                try:
                    df['volesti']['details'] = df['volesti']['details'].apply(literal_eval)
                                        
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['output'] for i in x]))
                    df['volesti']['output'] = list(zip(df['volesti']['np'].apply(np.mean), df['volesti']['np'].apply(np.std)))            
                
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['time'] for i in x]))
                    df['volesti']['time'] = df['volesti']['np'].apply(np.mean)            
                
                except:
                    pass
            
    except Exception as e:
        pass

/home/des/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/des/venv/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/des/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
tools = sorted(list(df.keys()))
tool0 = tools[0]
tool1 = tools[1]

df_res = df[tool0].join(df[tool1], on='index', lsuffix=f'_{tool0}', rsuffix=f'_{tool1}', how='outer')

for t in tools[2:]:
    df_res = df_res.join(df[t].add_suffix(f'_{t}'), on='index', how='outer')


In [ ]:
df_res = df_res.sort_values('index_gubpi')
selected_cols = [f'formula_gubpi']

for t in tools:
    selected_cols.append(f"output_{t}")
    df_res[f"output_{t}"] = df_res[f"output_{t}"].astype(str).str[:1000]
    selected_cols.append(f"time_{t}")
    df_res[f"time_{t}"] = df_res[f"time_{t}"].fillna(0).apply(lambda x:round(x, 3))
    
# df_res.columns
df_res = df_res[selected_cols]
df_res.head()

,formula_gubpi,output_faza,time_faza,output_gubpi,time_gubpi,output_latte,time_latte,output_psi,time_psi,output_volesti,time_volesti
0.0,(x)*1,"(0.4650292968749998, 0.5641259765624999)",1.886,"(0.4874999999999982, 0.5125000000000021)",0.341,0.49995,0.076,E[p_] = 1/2,0.017,"(0.49807700000000005, 0.027256533029716014)",0.058
1.0,(x**2)*1,"(0.291015625, 0.390625)",2.923,"(0.3331775037124986, 0.33348919462330495)",0.441,0.3333333333333333,0.101,E[p_] = 1/3,0.033,"(0.32744969999999995, 0.01718050319431885)",0.068
2.0,(x**3)*1,"(0.201171875, 0.30078125)",15.674,"(0.24984827016353228, 0.25015176909508047)",0.426,0.25,0.071,E[p_] = 1/4,0.029,"(0.24845979999999995, 0.012861644403419028)",0.068
3.0,(x**3 + x**2 + x + 1)*1,"(2.0322265625, 2.1318359375)",30.070,"(2.0804081595527166, 2.086258920654905)",0.470,2.083333333333333,0.100,E[p_] = 25/12,0.026,"(2.064878, 0.08109399445088401)",0.064
4.0,(x**4 + x**3 + x**2 + x**1 + 1)*1,"(2.23297119140625, 2.3326873779296875)",185.701,"(1.7791781814430736, 2.2864838808790706)",0.449,2.283333333333333,0.087,E[p_] = 137/60,0.027,"(2.3449210000000003, 0.09601009118316675)",0.062


In [214]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)